In [1]:
!pip install osmnx h3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.4/101.4 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 8.3 MB/s eta 0:00:00


In [10]:
import h3
import osmnx as ox
import networkx as nx
import geopandas as gpd
import pandas as pd
import numpy as np
from shapely import wkt
from functools import reduce
from datetime import datetime
from typing import Optional
from shapely.geometry import Polygon

In [22]:
nodes = pd.read_csv("data/external/GrafoLima_nodes.csv", index_col=[0])
nodes["geometry"] = nodes["geometry"].apply(wkt.loads)
nodes = gpd.GeoDataFrame(nodes, crs="epsg:4326")
nodes

,y,x,ref,highway,street_count,geometry
osmid,,,,,,
258066779,-12.371360,-76.784678,45A,motorway_junction,3,POINT (-76.78468 -12.37136)
258066780,-12.376318,-76.779559,NaN,NaN,3,POINT (-76.77956 -12.37632)
258066781,-12.384553,-76.771046,NaN,NaN,3,POINT (-76.77105 -12.38455)
258066783,-12.385783,-76.769768,NaN,NaN,3,POINT (-76.76977 -12.38578)
258067237,-12.304254,-76.840727,NaN,NaN,3,POINT (-76.84073 -12.30425)
...,...,...,...,...,...,...
11074106301,-11.822343,-77.161338,NaN,NaN,4,POINT (-77.16134 -11.82234)
11074106303,-11.821708,-77.161237,NaN,NaN,4,POINT (-77.16124 -11.82171)
11074106305,-11.821329,-77.161223,NaN,NaN,3,POINT (-77.16122 -11.82133)


In [23]:
edges = pd.read_csv("data/external/GrafoLima_edgeswaze.csv", index_col=[0,1,2])
edges["geometry"] = edges["geometry"].apply(wkt.loads)
edges = gpd.GeoDataFrame(edges, crs="epsg:4326")
edges

<ipython-input-23-a8c8cb5220b6>:1: DtypeWarning: Columns (5,7,13,14,15,17) have mixed types. Specify dtype option on import or set low_memory=False.
  edges = pd.read_csv("data/external/GrafoLima_edgeswaze.csv", index_col=[0,1,2])


osmid  oneway lanes  \
u           v           key                                                     
258066779   10985400003 0                              39457621    True     1   
            258066780   0    [1046533254, 338193454, 143630423]    True     2   
258066780   258066781   0                             143630430    True     2   
258066781   258066783   0                             143630430    True     2   
            495302399   0                              40752940    True     1   
...                                                         ...     ...   ...   
11074106305 11074106303 0                            1193021154   False   NaN   
11074106311 5497647609  0                             439736677   False   NaN   
11074106403 1273703683  0                             111938577   False   NaN   
            11001190815 0                             111931967   False   2.0   
            10738000177 0                             111938577   False   NaN   

                                   highway maxspeed    length  \
u           v           key                                     
258066779   10985400003 0    motorway_link       80   109.492   
            258066780   0         motorway      100   782.922   
258066780   258066781   0            trunk      100  1301.315   
258066781   258066783   0            trunk      100   194.871   
            495302399   0       trunk_link      NaN    85.154   
...                                    ...      ...       ...   
11074106305 11074106303 0      residential      NaN    42.177   
11074106311 5497647609  0      residential      NaN   126.496   
11074106403 1273703683  0      residential      NaN    51.814   
            11001190815 0      residential      NaN    61.585   
            10738000177 0      residential      NaN   113.466   

                                                                      geometry  \
u           v           key                                                      
258066779   10985400003 0    LINESTRING (-76.78468 -12.37136, -76.78466 -12...   
            258066780   0    LINESTRING (-76.78468 -12.37136, -76.78433 -12...   
258066780   258066781   0    LINESTRING (-76.77956 -12.37632, -76.77670 -12...   
258066781   258066783   0    LINESTRING (-76.77105 -12.38455, -76.76977 -12...   
            495302399   0    LINESTRING (-76.77105 -12.38455, -76.77099 -12...   
...                                                                        ...   
11074106305 11074106303 0    LINESTRING (-77.16122 -11.82133, -77.16124 -11...   
11074106311 5497647609  0    LINESTRING (-77.15013 -11.82304, -77.15002 -11...   
11074106403 1273703683  0    LINESTRING (-77.16524 -11.77607, -77.16571 -11...   
            11001190815 0    LINESTRING (-77.16524 -11.77607, -77.16521 -11...   
            10738000177 0    LINESTRING (-77.16524 -11.77607, -77.16487 -11...   

                               ref                        name bridge  \
u           v           key                                             
258066779   10985400003 0      NaN                         NaN    NaN   
            258066780   0    PE-1S  Carretera Panamericana Sur    yes   
258066780   258066781   0    PE-1S  Carretera Panamericana Sur    NaN   
258066781   258066783   0    PE-1S  Carretera Panamericana Sur    NaN   
            495302399   0      NaN                         NaN    NaN   
...                            ...                         ...    ...   
11074106305 11074106303 0      NaN                         NaN    NaN   
11074106311 5497647609  0      NaN               Calle Samiria    NaN   
11074106403 1273703683  0      NaN                         NaN    NaN   
            11001190815 0      NaN                         NaN    NaN   
            10738000177 0      NaN                         NaN    NaN   

                            junction access tunnel  width service  \
u           v           key                                         
258

In [24]:
graph = ox.graph_from_gdfs(gdf_nodes=nodes, gdf_edges=edges)
nodes = nodes.reset_index()
nodes.rename(columns={"y": "lat", "x": "lon"}, inplace=True)

In [25]:
districts = gpd.read_file("./data/external/DISTRITOS.shp")
districts = districts[districts["PROVINCIA"].isin(["LIMA", "CALLAO"])].explode(index_parts=True).reset_index(drop=True)

In [26]:
density = gpd.read_file("./data/external/density.gpkg")
density["hex_id"] = density["h3"]
density = density[["hex_id", "population"]].copy()
density.rename(columns={"population": "population_orig"}, inplace=True)

In [27]:
level = 8
hexagons = []

for geom in districts["geometry"]:
    hexagons.extend(h3.polyfill(geom.__geo_interface__, res=level, geo_json_conformant=True))

hexagon_geoms = [Polygon(h3.h3_to_geo_boundary(hexagon, geo_json=True)) for hexagon in hexagons]
hexagon_gdf = gpd.GeoDataFrame(geometry=hexagon_geoms)
hexagon_gdf["hex_id"] = hexagons
hexagon_gdf["center"] = hexagon_gdf["geometry"].apply(lambda x: x.centroid)
hexagon_gdf["lon"] = hexagon_gdf["center"].apply(lambda x: x.x)
hexagon_gdf["lat"] = hexagon_gdf["center"].apply(lambda x: x.y)
hexagon_gdf = hexagon_gdf[["hex_id", "lat", "lon"]]

origins = pd.merge(hexagon_gdf, density, on="hex_id", how="inner")
origins = origins[origins["population_orig"] >= 100]
origins.to_csv("./data/outputs/raw_origins.csv", index=False)

In [28]:
nearest_origins = ox.distance.nearest_nodes(graph, origins["lon"], origins["lat"], return_dist=False)
new_origins = nodes[nodes["osmid"].isin(nearest_origins)].reset_index(drop=True).copy()
new_origins["hex_id"] = new_origins.apply(
    lambda x: h3.geo_to_h3(x.get("lat"), x.get("lon"), 8),
    axis=1
)
new_origins = pd.merge(new_origins, density, on="hex_id", how="inner")
new_origins = new_origins[["osmid", "lon", "lat", "population_orig"]]
new_origins

,osmid,lon,lat,population_orig
0,258067461,-76.919109,-12.257607,1330.0
1,495229959,-76.871312,-12.214252,1353.0
2,495280445,-76.752538,-12.393217,213.0
3,513858708,-76.861323,-12.282351,939.0
4,799255264,-76.774448,-12.400708,235.0
...,...,...,...,...
1614,11028473022,-76.947774,-11.787134,245.0
1615,11066139345,-77.018269,-11.835892,2791.0
1616,11070465100,-76.947741,-11.868826,596.0
1617,11072483927,-77.162444,-11.757260,673.0


In [29]:
new_origins.to_csv("./data/outputs/origins.csv", index=False)